# PageRank

In [76]:
from databaseconnection import DatabaseConnection
gds = DatabaseConnection().get_database_connection()
gds.version()

'2.3.2'

In [3]:
gds.run_cypher("""
               CALL gds.graph.project(
                   'popularQuestionAlternatives',
                   ['QuestionAlternative', 'Respondent'],
                   {
                        CHOSE: {
                            orientation: 'REVERSE'
                        }
                   }
               )
               """)

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.project`: Caused by: java.lang.IllegalArgumentException: A graph with name 'popularQuestionAlternatives' already exists.}

In [4]:
gds.run_cypher("""
               CALL gds.degree.write.estimate('popularQuestionAlternatives', 
               { writeProperty: 'degree' })
               YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
               """)

,nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
0,5010,203940,56,56,56 Bytes


In [5]:
gds.run_cypher("""
               CALL gds.graph.relationshipProperties.write('popularQuestionAlternatives', {
                     relationshipProperties: {
                            CHOSE: {
                                property: 'score',
                                defaultValue: 1.0
                            }
                        }
                    })
               """)

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '': expected
  "!="
  "%"
  "*"
  "+"
  ","
  "-"
  "."
  "/"
  ":"
  "<"
  "<="
  "<>"
  "="
  "=~"
  ">"
  ">="
  "AND"
  "CONTAINS"
  "ENDS"
  "IN"
  "IS"
  "OR"
  "STARTS"
  "XOR"
  "["
  "^"
  "}" (line 8, column 16 (offset: 324))
""
                ^}

In [6]:
gds.run_cypher("""
               CALL gds.degree.mutate('popularQuestionAlternatives',
                { mutate })
                YIELD nodeId, score
                WHERE score > 0
                RETURN gds.util.asNode(nodeId).name AS name, score AS respondents
                ORDER BY respondents ASC, name ASC
               """)

Failed to read from defunct connection IPv4Address(('localhost', 7687)) (IPv4Address(('127.0.0.1', 7687)))


SessionExpired: Failed to read from defunct connection IPv4Address(('localhost', 7687)) (IPv4Address(('127.0.0.1', 7687)))

In [ ]:
gds.run_cypher("""
               CALL gds.degree.stats('popularQuestionAlternatives')
               YIELD centralityDistribution
               RETURN centralityDistribution.min as minimumScore, centralityDistribution.max as maximumScore, centralityDistribution.mean as meanScore
               """)


In [9]:
gds.run_cypher("""
               CALL gds.degree.stream(
                    'popularQuestionAlternatives'
               )
               YIELD nodeId, score
               RETURN gds.util.asNode(nodeId).name AS name, score AS respondents
               ORDER by respondents DESC, name ASC
               """)

,name,respondents
0,"Nej, det har inte hänt mig",2468.0
1,Sverige,2458.0
2,"Nej, det har inte hänt mig",2410.0
3,"Nej, det har inte hänt mig",2390.0
4,Jag har inte varit med och mobbat andra elever...,2387.0
...,...,...
5005,None,0.0
5006,None,0.0
5007,None,0.0
5008,None,0.0


In [11]:
gds.run_cypher("""
               CALL gds.degree.write(
                    'popularQuestionAlternatives',
                    { writeProperty: 'nrOfRespondents' }
               )
               YIELD nodePropertiesWritten
               RETURN nodePropertiesWritten
               """)

,nodePropertiesWritten
0,5010


In [15]:
# Create relationship from questionalternative to respondent called CHOSEN_BY if the there already is a relationship between the two nodes called CHOSE. 
# The relationship CHOSEN_BY will have a property called score which is the same as the property nrOfRespondents on the node QuestionAlternative.
gds.run_cypher("""
                MATCH (q:QuestionAlternative)<-[c:CHOSE]-(r:Respondent)
                MERGE (q)-[cb:CHOSEN_BY]->(r)
                SET cb.score = q.nrOfRespondents
               """)

""


In [21]:
# Create graph on the new relationship CHOSEN_BY

gds.run_cypher("""
               CALL gds.graph.project(
                     'outkastRespondents',
                     ["QuestionAlternative", "Respondent"],
                     {
                            CHOSEN_BY: { 
                                properties: 'nrOfRespondents'
                            }
                     }
                ) YIELD graphName, nodeCount, relationshipCount
                AS graph
                RETURN graph
               """)

,graph
0,203940


In [55]:
# Create relation HAS_PARTICIPATED between Respondent and Survey. 
# Currently Respondents have relation HAS_ANSWERED to Question which has 
# relation HAS_QUESTION to Survey. 

gds.run_cypher("""
                MATCH (r:Respondent)-[a:HAS_ANSWERED]->(q:Question)<-[h:HAS_QUESTION]-(s:Survey)
                MERGE (r)-[p:HAS_PARTICIPATED]->(s)
               """)

""


In [75]:
# Divide the property nrOfRespondents on the node QuestionAlternative
# by the number of Repondents with relation HAS_PARTICIPATED to the Survey
# that the Question with relation CONSISTS_OF to said QuestionAlternative
# belongs to. This will give us the percentage of Respondents that chose
# the QuestionAlternative in relation to the total number of Respondents.

#gds.run_cypher("""
#                MATCH (qa:QuestionAlternative)<-[ch:CHOSE]-(re:Respondent)-[hp:HAS_PARTICIPATED]->(su:Survey)-[hq:HAS_QUESTION]->(qu:Question)-[co:CONSISTS_OF]->(qua:QuestionAlternative)
#                WHERE qa.id=qua.id
#                return qa.nrOfRespondents / count(re) as percentage, qa.name as name, qa.nrOfRespondents as nrOfRespondents, count(re) as nrOfRespondentsInSurvey, su.name as surveyName
#                LIMIT 10;
#               """)

# The above gave this result:
# 	percentage	name	                    nrOfRespondents	nrOfRespondentsInSurvey	surveyName
# 0	1.000000	Stämmer mycket väl	        24.0	        24	                    Elevenkät Referensskola
# 1	0.500000	Nej, det har inte hänt mig	54.0	        108	                    Elevenkät Referensskola
# 
# nrOfRespondentsInSurvey is wrong since the both rows contain the same survey but nrOfRespondentsInSurvey is 24 and 108 respectively.
# Do a query where the nrOfRespondentsInSurvey actually is correct.

gds.run_cypher("""
                MATCH (su:Survey)-[:HAS_QUESTION]->(qu:Question)-[:CONSISTS_OF]->(qa:QuestionAlternative)<-[:CHOSE]-(re:Respondent)-[:HAS_PARTICIPATED]->(su)
                WITH su, count(DISTINCT re) as nrOfRespondentsInSurvey
                MATCH (su)-[:HAS_QUESTION]->(qu:Question)-[:CONSISTS_OF]->(qa:QuestionAlternative)<-[:CHOSE]-(re2:Respondent)-[:HAS_PARTICIPATED]->(su)
                WITH su, qa, nrOfRespondentsInSurvey, count(re2) as nrOfRespondents
                RETURN nrOfRespondents * 1.0 / nrOfRespondentsInSurvey as percentage, qa.name as name, nrOfRespondents as nrOfRespondents, nrOfRespondentsInSurvey, su.name as surveyName
                LIMIT 1000;
               """) 


,percentage,name,nrOfRespondents,nrOfRespondentsInSurvey,surveyName
0,0.024691,Stämmer mycket dåligt,2,81,Elevenkät Referensskola
1,0.358025,Stämmer ganska väl,29,81,Elevenkät Referensskola
2,0.061728,Stämmer ganska dåligt,5,81,Elevenkät Referensskola
3,0.037037,Vet inte / Kan inte svara,3,81,Elevenkät Referensskola
4,0.358025,Stämmer mycket väl,29,81,Elevenkät Referensskola
...,...,...,...,...,...
995,0.024349,Jag förstår inte frågan,72,2957,Elevenkät åk 4-6 pilot exkl sexuella trakasserier
996,0.070680,Vet inte/vill inte svara,209,2957,Elevenkät åk 4-6 pilot exkl sexuella trakasserier
997,0.189043,Båda föräldrarna är födda i ett annat land,559,2957,Elevenkät åk 4-6 pilot exkl sexuella trakasserier
998,0.656409,Båda födda i Sverige,1941,2957,Elevenkät åk 4-6 pilot exkl sexuella trakasserier


In [ ]:
gds.run_cypher("""
                MATCH (su:Survey)-[:HAS_QUESTION]->(qu:Question)-[:CONSISTS_OF]->(qa:QuestionAlternative)
                WITH su, qa, qa.nrOfRespondents as nrOfRespondents
                SET qa.percentageOfRespondents = nrOfRespondents * 1.0 / su.totalParticipants
                RETURN qa.name as questionAlternativeName, qa.percentageOfRespondents as percentageOfRespondents, su.name as surveyName;
                """)